# Profile and Statistics Plots for RRTMGP Band Generation

## Necessary Libraries

In [ ]:
# standard libraries
import os, sys
import datetime as DT

# pip install
import numpy as np
import matplotlib.pyplot as plot
from matplotlib import rc
import matplotlib.font_manager as font
import netCDF4 as nc
import time # for process_timeing processes

# for multi-page PDF files
from matplotlib.backends.backend_pdf import PdfPages

# Pernak Python module
LIBPATHS = ['common', 'RRTMGP-profile-stats-plots']
for path in LIBPATHS: sys.path.append(path)
import utils
from LBLRTM_RRTMGP_compare import * # profPDFs(), statPDF()
from CHARTS_RRTMGP_compare import * # refTestFluxCompare()

## main()

### Runtime Variables

These are variables that will likely need to be reassigned depending on the analysis to be done. We are using the default [plotting](https://github.com/AER-RC/RRTMGP-profile-stats-plots/blob/master/LBLRTM_RRTMGP_config.ini) [configurations](https://github.com/AER-RC/RRTMGP-profile-stats-plots/blob/master/LBLRTM_RRTMGP_compare.py#L1011) as the template for what inputs are passed to the plotting cell. If any of these variables are changed, this code cell will need to be re-run.

In [ ]:
doLW = False

domainStr = 'lw' if doLW else 'sw'

# paths to temporary testing files
PROJECT = '/project/projectdirs/e3sm/pernak18/reference_netCDF'

# input files; NOTE: NO FORCING HAS BEEN IMPLEMENTED IN THIS NOTEBOOK
# reference LBL file will likely not change; test RRTMGP file will point 
# to netCDFs written with g-point-reduction Notebook
refDir = '{}/g-point-reduce'.format(PROJECT)
testDir = '{}/profile-stats-plots'.format(PROJECT)

fluxSuffix = 'flux-inputs-outputs-garandANDpreind.nc'

# g-point reduction iteration number (unit-offset)
iterID = 0

if doLW:
    refFile = '{}/lblrtm-{}-{}'.format(refDir, domainStr, fluxSuffix)

    # reference 256-g-point RRTMGP
    testFile = '{}/rrtmgp-{}-flux-inputs-outputs-garand-all.nc'.format(
        testDir, domainStr)
    refFile = '{}/rrtmgp-{}-flux-inputs-outputs-garand-all.nc'.format(
        testDir, domainStr)
    refFile = '{}/lblrtm-{}-{}'.format(refDir, domainStr, fluxSuffix)
    testFile = '{}/rrtmgp-{}-{}'.format(refDir, domainStr, fluxSuffix)
    #testFile = 'optimized_fluxes_iter{}.nc'.format(iterID)
    testFile = '/global/homes/p/pernak18/RRTMGP/g-point-reduction/temp_iter128.nc'
else:
    refFile = '{}/charts-{}-{}'.format(refDir, domainStr, fluxSuffix)
    testFile = '{}/rrtmgp-{}-{}'.format(refDir, domainStr, fluxSuffix)
    testFile = '/global/cscratch1/sd/pernak18/RRTMGP/optimized_SW_fluxes_iter112.nc'
# endif doLW

# make sure paths exist
paths = [refFile, testFile]
for path in paths: utils.file_check(path)

# plotting parameters
plot_profiles = True
plot_stats = True
xt = 'LBLRTM'
yt = 'RRTMGP-LBLRTM'
aType = 'Garand'
log_y = True
single_stat = True

# unit-offset forcing scenario; dimension 1 in RRTMGP netCDFs
# https://github.com/pernak18/g-point-reduction/wiki/LW-Forcing-Number-Convention#g-point-reduction-convention-
# https://github.com/pernak18/g-point-reduction/wiki/SW-Number-Convention
IRECORD = 7
DOFORCE = True

# g-point reduction cost function, iteration number, and normalization ID
suffix = 'iter{:03d}_rec{:02d}'.format(iterID, IRECORD)

# prefixes for output files
profPrefix = 'PROFS_{}_LBLRTM_RRTMGP_{}'.format(domainStr, suffix)
statPrefix = 'stats_{}_lblrtm_rrtmgp_{}'.format(domainStr, suffix)

# pressure at tropopause (mbar)
pTrop = np.exp(4.6)

# unit-offset number of band to plot
# can also be "None" (all bands) or [] (no single bands)
bands = []

# convert to zero-offset band numbers; None ==> all bands
#inBand = None if bands is None else np.array(bands)-1
doBroadband = True

# some additional processing needs to be done for forcing -- 
# new netCDFs need to be generated
if DOFORCE:
    PIPPATH = '{}/.local/'.format(os.path.expanduser('~')) + \
        'cori/3.7-anaconda-2019.10/lib/python3.7/site-packages'
#     PIPPATH = '/global/homes/e/emlawer/.local/cori/3.8-anaconda-2020.11/' + \
#         'lib/python3.8/site-packages'

    sys.path.append(PIPPATH)
    import xarray as xa

    # newer RRTMGP-convention netCDFs for this project have forcing 
    # scenarios consolidated into a single file, but the older plotting 
    # programs expect separate files for each forcing scenario, so we 
    # will create those files in this cell
    iRec = IRECORD - 1
    with xa.open_dataset(refFile) as rDS, xa.open_dataset(testFile) as tDS:
        # baseline (preindustrial) and forcing datasets
        iBase = 1 if iRec < 7 else 0
        if doLW:
            fluxVars = ['band_flux_net', 'band_flux_up', 'band_flux_dn', \
              'band_heating_rate', 'flux_net', 'flux_up', 'flux_dn', \
              'heating_rate']
        else:
            # endif iForce
            fluxVars = ['band_flux_net', 'band_flux_up', 'band_flux_dn', \
              'band_heating_rate', 'flux_net', 'flux_up', 'flux_dn', \
              'heating_rate', 'band_flux_dir_dn', 'flux_dir_dn']
        # endif doLW

        allDS = [rDS.isel(record=iBase), tDS.isel(record=iBase), 
                 rDS.isel(record=iRec), tDS.isel(record=iRec)]
    # endwith

    allNames = ['lblrtm-lw-flux-inputs-outputs-garand.nc', 
                'rrtmgp-lw-flux-inputs-outputs-garand.nc', 
                'lblrtm-lw-flux-inputs-outputs-garand-forcing.nc', 
                'rrtmgp-lw-flux-inputs-outputs-garand-forcing.nc']

    for ds, name in zip(allDS, allNames):
        ds.attrs['model-description'] = ''
        ds.to_netcdf(name)
    # end ds loop

    
    refFile, testFile = forcingDiff(
        allNames[0], allNames[1], allNames[2], allNames[3], repVars=fluxVars)
# endif DOFORCE

# Instantiate a SW object needed for its plotting
# likely no editing needed here, but the block
# is dependent on variables that have been changed 
if not doLW:
    # SW plotting with reference CHARTS results were done with OOD
    configDict = {
        'ref': refFile, 'test': testFile, \
        'x': 'CHARTS', 'y': 'RRTMGP-CHARTS', \
        'atm': 'Garand', 'forcing': False, \
        'stat': statPrefix, 'prof': profPrefix
    }

    iRec = IRECORD-1
    if iRec == 18: iRec -= 11

    plotObj = refTestFluxCompare(configDict, outDir='.', \
        tPauseP=pTrop, bands=bands, record=iRec, \
        yLog=log_y, broadband=doBroadband, convertHR=True)
# endif LW

Now we'll proceed with profile plot generation. The process is time consuming, but the progress -- for netCDF reading, band number, column number -- is printed to standard output (i.e., in this notebook, after the code cell). Once the figures are saved, they can be opened in a separate Jupyter tab by clicking on them in the left sidebar (the names follow the convention `plotPrefix_??.pdf`, where `plotPrefix` was defined in the previous "Runtime Variables" code cell, and `??` is the zero-padded band number, which can also be `broadband` if `doBroadband` is set to `True`). Additionally, each page of the PDF is rendered in this notebook. Figures can be deleted from the **notebook** by right-clicking on the output and selecting the "Clear Outputs" pull down option.

In [ ]:
if plot_profiles:
    # plot the test and reference upward and downward flux together
    # with the heating rates (and differences for each)
    if doLW:
        pBands = np.arange(16) if bands is None else np.array(bands)-1

        for iBand in pBands:
            profPDFs(refFile, testFile, yt, tPauseP=pTrop, \
              prefix=profPrefix, atmType=aType, inBand=iBand, \
              yLog=log_y, hrUnits=True, iForce=IRECORD)
        # end iBand loop

        # for specified bands AND broadband
        if doBroadband:
            profPDFs(refFile, testFile, yt, tPauseP=pTrop, \
              prefix=profPrefix, atmType=aType, \
              yLog=log_y, broadOnly=doBroadband, hrUnits=True, 
              iForce=IRECORD)
        # end broadband plot
    else:
        # workaround for broadband and by-band plots -- i need 
        # to fix the bug where only BB is plotted if doBroadband is True
        if doBroadband: plotObj.profPDF()
        plotObj.broadBand = False
        plotObj.profPDF()
    # endif LW
# end plot_profiles

In [ ]:
# plot profile statistics
if plot_stats:
    if doLW:
        statPDF(refFile, testFile, singlePDF=single_stat, \
          tPauseP=pTrop, xTitle=xt, yTitle=yt, prefix=statPrefix, \
          atmType=aType, statCSV='', forcing=False, hrUnits=True, 
          iRecord=IRECORD)
    else:
        plotObj.statPDF()
    # endif LW
# endif plot_stats

A similar procedure is performed for the statistics plots, which are less time consuming to produce. Because of the smaller amount of time needed for the statistics, they are calculated for all bands and broadband. Again, status is written to standard output, and files can be accessed in the left sidebar. File names conform to the `statPrefix_??.pdf` convention, where `statPrefix` is defined in the "Runtime Variables" code cell, and `??` is the zero-padded band number or `broadband`. Plots are also rendered in standard output. Figures can be deleted from the **notebook** by right-clicking on the output and selecting the "Clear Outputs" pull down option.